# Preprocess data from batdongsan website

In [33]:
%pwd

'd:\\OneDrive - Hanoi University of Science and Technology\\HUST\\CLASS\\20211\\DataScience\\Project\\Test_alonhadat'

In [2]:
%cd ..

d:\OneDrive - Hanoi University of Science and Technology\HUST\CLASS\20211\DataScience\Project\Test_alonhadat


In [3]:
import findspark
findspark.init()

import pyspark
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark.sql.types import *
import numpy as np 



In [4]:
spark = SparkSession.builder.appName('Clean Data').getOrCreate()

In [5]:
folder = 'resources/data/final/'
files = [ "bds_lat_long.csv", "alo_nha_dat_lat_long.csv", "out_department_lat_long.csv", "out_house_lat_long.csv"]
files_path = [folder + file for file in files]

In [18]:
files_path

['resources/data/final/bds_lat_long.csv',
 'resources/data/final/alo_nha_dat_lat_long.csv',
 'resources/data/final/out_department_lat_long.csv',
 'resources/data/final/out_house_lat_long.csv']

In [6]:
schema = StructType([
    StructField('address', StringType(), True),
    StructField('facade', FloatType(), True),
    StructField('direction', StringType(), True),
    StructField('area', FloatType(), True),
    StructField('bedroom', IntegerType(), True),
    StructField('bathroom', IntegerType(), True),
    StructField('floors', IntegerType(), True),
    StructField('house_type', StringType(), True),
    StructField('price', FloatType(), True),
    StructField('street_size', FloatType(), True),
    StructField('lat', FloatType(), True),
    StructField('lng', FloatType(), True),
])



In [7]:
spark

In [8]:
bds_df = spark.read.csv("resources/data/final/bds_lat_long.csv", header=True)

## Convert Type

In [10]:
cvt = {"StringType" : 'str',
 "FloatType" : 'float',
 "IntegerType" : 'int'}

sql_cvt = [f"cast({field.name} as {cvt[str(field.dataType)]})  {field.name}" if str(field.dataType) != 'StringType' else f"{field.name}" for field in schema.fields ]

bds_df_cast = bds_df.selectExpr(*sql_cvt)

In [11]:
bds_df_cast.show()

+--------------------+------+---------+------+-------+--------+------+--------------------+-----+-----------+--------+---------+
|             address|facade|direction|  area|bedroom|bathroom|floors|          house_type|price|street_size|     lat|      lng|
+--------------------+------+---------+------+-------+--------+------+--------------------+-----+-----------+--------+---------+
| Nam Từ Liêm, Hà ...|   4.3| Đông-Nam|  40.0|      6|       4|     5|       Bán nhà riêng| 2.85|       null|21.00813| 105.7665|
| Dự án The Link 3...|  null|     null|  72.0|      2|       2|  null| Bán căn hộ chung cư| 2.88|       null|21.08525|105.78535|
| Dự án The Manor ...|  null|     null|  null|      4|       6|     5|Bán nhà mặt phố (...| 18.0|       null|20.98051|105.81113|
| Phố Liễu Giai, P...|   6.0|     null|  60.0|     12|    null|     7|Bán nhà mặt phố (...| 36.2|       40.0|21.03517|  105.815|
| Dự án Sunshine H...|  35.0|     null|1005.0|   null|    null|  null|Bán đất nền dự án...|12.06|

In [22]:
bds_df_cast.printSchema()

bds_df_cast.count()

root
 |-- address: string (nullable = true)
 |-- facade: float (nullable = true)
 |-- direction: string (nullable = true)
 |-- area: float (nullable = true)
 |-- bedroom: integer (nullable = true)
 |-- bathroom: integer (nullable = true)
 |-- floors: integer (nullable = true)
 |-- house_type: string (nullable = true)
 |-- price: float (nullable = true)
 |-- street_size: float (nullable = true)
 |-- lat: float (nullable = true)
 |-- lng: float (nullable = true)



29612

## Remove some null value

In [26]:
# remove null address, price or lat/lng

bds_df_cast = bds_df_cast.filter("address is not NULL")\
    .filter("price is not NULL")\
    .filter("lat != 0.0")\
    .filter("area is not NULL")
    
print("Number of rows after filter: ", bds_df_cast.count())

Number of rows after filter:  24745


In [27]:
bds_df_cast.describe().show()

+-------+--------------------+-----------------+---------+------------------+-----------------+-----------------+------------------+--------------------+----------------+------------------+------------------+------------------+
|summary|             address|           facade|direction|              area|          bedroom|         bathroom|            floors|          house_type|           price|       street_size|               lat|               lng|
+-------+--------------------+-----------------+---------+------------------+-----------------+-----------------+------------------+--------------------+----------------+------------------+------------------+------------------+
|  count|               24745|            13970|     7065|             24745|            16136|            13486|             13203|               24745|           24745|             11495|             24745|             24745|
|   mean|                null|7.239799565627222|     null|249.45406749591993|4.441001487

In [32]:
bds_df_cast.toPandas().to_csv("resources/data/final/clean/bds_lat_long_clean.csv", header=True)